# Data ingestion to Bronze

In [0]:
%sql
create connection if not exists Databricks_earthquake
type HTTP
OPTIONS
(
  host = 'https://earthquake.usgs.gov',
  port = 443,
  base_path = '/earthquakes/feed/v1.0/',
  bearer_token = 'na'
)

In [0]:
from databricks.sdk import WorkspaceClient
w = WorkspaceClient()
conn = w.connections.get("Databricks_earthquake")
base_url = f"{conn.options['host']}{conn.options['base_path']}"

In [0]:
dbutils.widgets.text('catalog_name','')
catalog_name = dbutils.widgets.get('catalog_name')


In [0]:
spark.sql(
    f"use catalog {catalog_name}"
    )
spark.sql(
    "use schema bronze_dev"
    )
spark.sql(
    "create volume if not exists earthquake_data"
    )

In [0]:
import requests
import json, datetime

url = f"{base_url}/summary/all_day.geojson"
response = requests.get(url)
if response.status_code != 200:
    raise Exception(f"Error in getting data from {url}")
data = response.json()
current_date = datetime.datetime.now().strftime("%Y-%m-%d")
dbutils.fs.put(
    f"/Volumes/{catalog_name}/bronze_dev/earthquake_data/earthquake_data_{current_date}.lson",
    json.dumps(data),
    overwrite=True,
)